In [7]:
import numpy as np
import matplotlib.pyplot as plt
import librosa as lb
from scipy.fftpack import fft
import sounddevice as sd
from scipy.signal import hann, hamming

from audiolazy import lazy_lpc as lpc

ImportError: cannot import name 'Sequence' from 'collections' (c:\Users\Mihai\anaconda3\Lib\collections\__init__.py)

In [ ]:
# provided functions
def est_predictor_gain(x, a, p):
    '''
    A function to compute gain of the residual signal in LP analysis.
    x:  signal 
    a: LPC coefficients
    p: order of the filter
    '''
    cor = np.correlate(x, x, mode='full')
    
    rr = cor[len(cor)//2: len(cor)//2+p+1]
    g = np.sqrt(np.sum(a*rr))
    return g

   
    
def reject_outliers(data, m=2):
    '''
    Function to reject outliers. All values beyond m standard deviations from mean are excluded
    '''
    return data[abs(data - np.mean(data)) < m * np.std(data)]

### 1. Linear prediction analysis. (1 point) 

In [1]:
# Read audio
audioIn, fs = lb.load('audio.wav', sr=None)   

# filter order
p = 4  # d) has to be tuned
p_values = [4, 8, 12]
# number of DFT points
nfft = 1024

inInd =0
wLen = int(0.02*fs) # 20 ms window
hop_length = int(wLen / 2)  # 50% overlap
win = hamming(wLen)

cnt = 0
num_frames = int(np.ceil((len(audioIn) - wLen) / hop_length))
formants_per_order = {p: [] for p in p_values}

# choose a representative frame of the vowel
plot_frame_idx = num_frames // 2

for p in p_values:
    inInd = 0
    cnt = 0
    
    while inInd < len(audioIn) - wLen:
        # Extract frame
        frame = audioIn[inInd:inInd + wLen] * win

        # a) Compute LPC coefficients and gain
        lpc_result = lpc.lpc.autocor(frame, p)
        a = np.array(lpc_result.numerator)
        gain = est_predictor_gain(frame, a, p)

        # b) Compute filter transfer function
        w, h = freqz(gain, a, worN=nfft, fs=fs)

        # c) Select a frame and plot FFT and LPC spectra
        if cnt == plot_frame_idx:
            # Compute FFT of the frame
            fft_spectrum = np.abs(fft(frame, nfft))[:nfft//2]
            fft_db = 20 * np.log10(fft_spectrum)
            
            lpc_spectrum = np.abs(h)
            lpc_db = 20 * np.log10(lpc_spectrum)

            plt.figure()
            plt.plot(np.linspace(0, fs / 2, len(fft_db)), fft_db, label="FFT Spectrum (dB)")
            plt.plot(w, lpc_db, label=f"LPC Spectrum (dB, Order {p})")
            plt.title(f"FFT vs LPC Spectrum (Order {p})")
            plt.xlabel("Frequency (Hz)")
            plt.ylabel("Magnitude (dB)")
            plt.legend()
            plt.show()

        # Formant estimation
        roots = np.roots(a)
        formant_freqs = np.angle(roots[np.imag(roots) >= 0]) * (fs / (2 * np.pi))
        valid_formants = formant_freqs[formant_freqs > 90]  # Filter out low frequencies
        formants_per_order[p].append(valid_formants[:3])  # First three formants
        
        inInd += hop_length
        cnt += 1
        

NameError: name 'lb' is not defined

#### d) Comment of the connection between lpc model order and peaks in the DFT spectrum.

#### Is the best order selection dependent on the vowel or not?

### 2. Formant analysis. (1 point)

#### Add more steps in the previous analysis loop

In [ ]:
# Read audio
audioIn, fs = lb.load('audio.wav', sr=None)   

# filter order
p = 4  # d) has to be tuned

# number of DFT points
nfft = 1024

inInd =0
wLen = int(0.02*fs) # 20 ms window
win = hamming(wLen) # hamming window for example

cnt = 0
numframes = np.ceil( (len(audioIn)-wLen)/(wLen/2)) # number of franes 
formants  = []                                     # A placeholder for storing formants

# choose a representative frame of the vowel
plot_frame = int(numframes/2)  # middle of the vowel

# The analysis loop
while inInd< len(audioIn)-wLen:
    # audio frame
    frame = audioIn[inInd:inInd+wLen]* win
    
    
    # Compute LPC and gain using audiolazy library
    
 
    # Compute the filter tansfer function using scipy.signal.freqz
 
    
    # Compute DFT spectrum
    
    
    # a) Compute roots (np.roots)
       
 
    # LPC coefficients are real-valued, the roots occur in complex conjugate pairs.
    # Retain only the roots with +ve sign for the imaginary part 
    
    
    # b.1) compute formants from roots (np.angle)
    

    # convert to Hertz from angular frequencies
    angz = angz*(fs/(2*np.pi))

    # sort the formants in increasing order
    angz = np.sort(angz)
    
    # remove zero frequencies
    angz = angz[angz !=0]
    
    # b.2) Save first three formants
    formants.append(angz[:3]) 
    
    inInd = inInd + int(wLen/2) # frame advance
       

        

formants = np.array(formants)
# b.3) For each separate vowel calculate average formant frequencies
print('------ The computed formants are :', np.mean(formants, 0))

# Refine formant estimations (optional)

#### b.4) Compare the obtained values with the indicative values. 

### Bonus problem: What happens if you apply LPC to the oboe sound? What are the corresponding "formants"?